## Imports

In [1]:
from pydantic import BaseModel

from aoc_utilities import puzzle_input

<h2 id="part2">--- Part One ---</h2>
Now, you need to figure out how to pilot this thing.

It seems like the submarine can take a series of commands like forward 1, down 2, or up 3:

    forward X increases the horizontal position by X units.
    down X increases the depth by X units.
    up X decreases the depth by X units.

Note that since you're on a submarine, down and up affect your depth, and so they have the opposite result of what you might expect.

The submarine seems to already have a planned course (your puzzle input). You should probably figure out where it's going. For example:

```
forward 5
down 5
forward 8
up 3
down 8
forward 2
```

Your horizontal position and depth both start at 0. The steps above would then modify them as follows:

    forward 5 adds 5 to your horizontal position, a total of 5.
    down 5 adds 5 to your depth, resulting in a value of 5.
    forward 8 adds 8 to your horizontal position, a total of 13.
    up 3 decreases your depth by 3, resulting in a value of 2.
    down 8 adds 8 to your depth, resulting in a value of 10.
    forward 2 adds 2 to your horizontal position, a total of 15.

After following these instructions, you would have a horizontal position of 15 and a depth of 10. (Multiplying these together produces 150.)

Calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

## Solution

In [2]:
class Instruction(BaseModel):
    direction: str
    units: int

    @staticmethod
    def parse(instruction_string):
        direction, units = instruction_string.split(' ')
        return Instruction(direction=direction, units=units)


class Submarine(BaseModel):
    horizontal_position: int = 0
    depth: int = 0

    class Config:
        allow_mutation = False

    def forward(self, units) -> 'Submarine':
        return Submarine(
            horizontal_position=self.horizontal_position + units,
            depth=self.depth
        )

    def up(self, units) -> 'Submarine':
        return Submarine(
            horizontal_position=self.horizontal_position,
            depth=self.depth - units
        )

    def down(self, units) -> 'Submarine':
        return Submarine(
            horizontal_position=self.horizontal_position,
            depth=self.depth + units
        )


def dive(
    submarine: Submarine = Submarine(),
    instructions_text: str = puzzle_input.as_text(day=2)
) -> Submarine:
    instructions = (
        Instruction.parse(instruction_string)
        for instruction_string in instructions_text.split('\n')
    )

    *_, submarine = (
        submarine := getattr(submarine, instruction.direction)(instruction.units)
        for instruction in instructions
    )
    return submarine

### Testing Our `Submarine`

In [3]:
example_instructions = '\n'.join([
    'forward 5',
    'down 5',
    'forward 8',
    'up 3',
    'down 8',
    'forward 2'
])

example_submarine = dive(instructions_text=example_instructions)

assert example_submarine.horizontal_position == 15
assert example_submarine.depth == 10

### Answer

In [4]:
submarine = dive()

print(
    'Submarine Location:'.ljust(32), submarine)
print(
    'Horizontal Position x Depth:'.ljust(32),
    submarine.horizontal_position * submarine.depth
)

Submarine Location:              horizontal_position=1909 depth=655
Horizontal Position x Depth:     1250395


<h2 id="part2">--- Part Two ---</h2><p>Based on your calculations, the planned course doesn't seem to make any sense. You find the submarine manual and discover that the process is actually slightly more complicated.</p>
<p>In addition to horizontal position and depth, you'll also need to track a third value, <em>aim</em>, which also starts at <code>0</code>. The commands also mean something entirely different than you first thought:</p>
<ul>
<li><code>down X</code> <em>increases</em> your aim by <code>X</code> units.</li>
<li><code>up X</code> <em>decreases</em> your aim by <code>X</code> units.</li>
<li><code>forward X</code> does two things:<ul>
  <li>It increases your horizontal position by <code>X</code> units.</li>
  <li>It increases your depth by your aim <em>multiplied by</em> <code>X</code>.</li>
</ul></li>
</ul>
<p>Again note that since you're on a submarine, <code>down</code> and <code>up</code> do the opposite of what you might expect: "down" means aiming in the positive direction.</p>
<p>Now, the above example does something different:</p>
<ul>
<li><code>forward 5</code> adds <code>5</code> to your horizontal position, a total of <code>5</code>. Because your aim is <code>0</code>, your depth does not change.</li>
<li><code>down 5</code> adds <code>5</code> to your aim, resulting in a value of <code>5</code>.</li>
<li><code>forward 8</code> adds <code>8</code> to your horizontal position, a total of <code>13</code>. Because your aim is <code>5</code>, your depth increases by <code>8*5=40</code>.</li>
<li><code>up 3</code> decreases your aim by <code>3</code>, resulting in a value of <code>2</code>.</li>
<li><code>down 8</code> adds <code>8</code> to your aim, resulting in a value of <code>10</code>.</li>
<li><code>forward 2</code> adds <code>2</code> to your horizontal position, a total of <code>15</code>.  Because your aim is <code>10</code>, your depth increases by <code>2*10=20</code> to a total of <code>60</code>.</li>
</ul>
<p>After following these new instructions, you would have a horizontal position of <code>15</code> and a depth of <code>60</code>. (Multiplying these produces <code><em>900</em></code>.)</p>
<p>Using this new interpretation of the commands, calculate the horizontal position and depth you would have after following the planned course. <em>What do you get if you multiply your final horizontal position by your final depth?</em></p>

## Solution

In [5]:
class AimingSubmarine(BaseModel):
    horizontal_position: int = 0
    depth: int = 0
    aim: int = 0

    class Config:
        allow_mutation = False

    def forward(self, units) -> 'AimingSubmarine':
        return AimingSubmarine(
            horizontal_position=self.horizontal_position + units,
            depth=self.depth + units * self.aim,
            aim=self.aim
        )

    def up(self, units) -> 'AimingSubmarine':
        return AimingSubmarine(
            horizontal_position=self.horizontal_position,
            depth=self.depth,
            aim=self.aim - units
        )

    def down(self, units) -> 'AimingSubmarine':
        return AimingSubmarine(
            horizontal_position=self.horizontal_position,
            depth=self.depth,
            aim=self.aim + units
        )

### Testing Our `AimingSubmarine`

In [6]:
example_instructions = '\n'.join([
    'forward 5',
    'down 5',
    'forward 8',
    'up 3',
    'down 8',
    'forward 2'
])

example_submarine = dive(AimingSubmarine(), example_instructions)

assert example_submarine.horizontal_position == 15
assert example_submarine.depth == 60

### Answer

In [7]:
submarine = dive(AimingSubmarine())

print(
    'Submarine Location:'.ljust(32), submarine)
print(
    'Horizontal Position x Depth:'.ljust(32),
    submarine.horizontal_position * submarine.depth
)

Submarine Location:              horizontal_position=1909 depth=760194 aim=655
Horizontal Position x Depth:     1451210346


# Comments

I didn't wind up taking advantage of Pydantic here, so pulling it in didn't really do me any favors.
Forcing faux immutability also meant more verbose code.
If there was an actual need to maintain multiple submarine classes, an abstract base class would fit nicely into this solution, with or without Pydantic and immutability.

In [8]:
from abc import ABC, abstractmethod


class Submarine(BaseModel, ABC):
    horizontal_position: int = 0
    depth: int = 0
    aim: int = 0

    @abstractmethod
    def forward(self, units: int) -> 'Submarine': ...

    @abstractmethod
    def up(self, units: int) -> 'Submarine': ...

    @abstractmethod
    def down(self, units: int) -> 'Submarine': ...

    def dive(self, instructions_text: str) -> Submarine:
        instructions = (
            Instruction.parse(instruction_string)
            for instruction_string in instructions_text.split('\n')
        )
        submarine = self

        *_, submarine = (
            submarine := getattr(submarine, instruction.direction)
                                (instruction.units)
            for instruction in instructions
        )
        return submarine


class VerticalDiveSubmarine(Submarine):
    def forward(self, units: int) -> 'VerticalDiveSubmarine':
        self.horizontal_position += units
        return self

    def up(self, units: int) -> 'VerticalDiveSubmarine':
        self.depth -= units
        return self

    def down(self, units: int) -> 'VerticalDiveSubmarine':
        self.depth += units
        return self


class DiagonalDiveSubmarine(Submarine):
    def forward(self, units) -> 'DiagonalDiveSubmarine':
        self.horizontal_position += units
        self.depth += units * self.aim
        return self

    def up(self, units) -> 'DiagonalDiveSubmarine':
        self.aim -= units
        return self

    def down(self, units) -> 'DiagonalDiveSubmarine':
        self.aim += units
        return self


vertical_dive_submarine = VerticalDiveSubmarine().dive(puzzle_input.as_text(day=2))
diagonal_dive_submarine = DiagonalDiveSubmarine().dive(puzzle_input.as_text(day=2))


print(
    'Vertical Dive Submarine Location:'.ljust(36),
    vertical_dive_submarine
)
print(
    'Horizontal Position x Depth:'.ljust(36),
    vertical_dive_submarine.horizontal_position * vertical_dive_submarine.depth
)

print(
    'Diagonal Dive Submarine Location:'.ljust(36), diagonal_dive_submarine
)
print(
    'Horizontal Position x Depth:'.ljust(36),
    diagonal_dive_submarine.horizontal_position * diagonal_dive_submarine.depth
)

Vertical Dive Submarine Location:    horizontal_position=1909 depth=655 aim=0
Horizontal Position x Depth:         1250395
Diagonal Dive Submarine Location:    horizontal_position=1909 depth=760194 aim=655
Horizontal Position x Depth:         1451210346
